In [11]:
import os

In [12]:
%pwd

'd:\\Nikku\\End to End Project\\Healthy.ai\\research\\Food image Classification'

In [15]:
os.chdir("../")

In [36]:
%pwd

'd:\\Nikku\\End to End Project\\Healthy.ai'

## Update the entity

In [37]:
from dataclasses import dataclass
from pathlib import Path



@dataclass(frozen=True)
class FoodImageDataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [38]:
from Healthyai.utils.common import read_yaml, create_directories
from Healthyai.constants import *

##  Configuration Manager In Src Config

In [39]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params =  read_yaml(params_filepath)
        
        create_directories([self.config.artifacts])
        
        
    #data ingestion for food image classification
       
    def get_food_data_ingestion(self) -> FoodImageDataIngestionConfig:
       config = self.config.food_image_data_ingestion
       
       create_directories([config.root_dir])
       
       food_image_data_ingestion_config = FoodImageDataIngestionConfig(
           root_dir = config.root_dir,
           source_URL = config.source_URL,
           local_data_file = config.local_data_file,
           unzip_dir = config.unzip_dir
           
       )
       return food_image_data_ingestion_config
        

## Update the Components

In [40]:
import os
import zipfile
import gdown
from Healthyai import logger
from Healthyai.utils.common import get_size

In [41]:
class FoodDataIngestion:
    def __init__(self, config: FoodImageDataIngestionConfig):
        self.config = config
        
    
    def download_data(self) -> str:
        """
        Fetch data from the Url
        """
        try:
            dataset_url =  self.config.source_URL
            zip_download_dir  = self.config.local_data_file
            os.makedirs("artifacts/food_image_classification/data_ingestion",exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")
            
            
            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)
            
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        """
        Zip_file_path: str
        Extract the zip file into the data directory
        Function retruns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

## Update the Pipeline

In [42]:
try:
    config = ConfigurationManager()
    food_image_data_ingestion = config.get_food_data_ingestion()
    food_image_data_ingestion = FoodDataIngestion(config=food_image_data_ingestion)
    food_image_data_ingestion.download_data()
    food_image_data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e


[2023-12-09 16:14:01,388: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-09 16:14:01,390: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-09 16:14:01,391: INFO: common: Created directory at : artifacts]
[2023-12-09 16:14:01,392: INFO: common: Created directory at : artifacts/food_image_classification/data_ingestion]
[2023-12-09 16:14:01,393: INFO: 261291339: Downloading data from https://drive.google.com/file/d/1lky7C3LipwPKv46QEjpTVHDTfrxEtXew/view?usp=drive_link into file artifacts/food_image_classification/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1lky7C3LipwPKv46QEjpTVHDTfrxEtXew
From (redirected): https://drive.google.com/uc?/export=download&id=1lky7C3LipwPKv46QEjpTVHDTfrxEtXew&confirm=t&uuid=629b0ab1-c5dd-4c26-bd80-6d82875e12e8
To: d:\Nikku\End to End Project\Healthy.ai\artifacts\food_image_classification\data_ingestion\data.zip
100%|██████████| 544M/544M [00:53<00:00, 10.2MB/s] 

[2023-12-09 16:14:56,181: INFO: 261291339: Downloaded data from https://drive.google.com/file/d/1lky7C3LipwPKv46QEjpTVHDTfrxEtXew/view?usp=drive_link into file artifacts/food_image_classification/data_ingestion/data.zip]
